In [22]:
import os
import json
import math
import numpy as np
import copy
from scipy.spatial.transform import Rotation

In [23]:
SAVE_PATH = os.path.join(os.getcwd(), '../data/jackal_evaluation_data')
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)
if not os.path.exists(os.path.join(SAVE_PATH, 'images')):
    os.makedirs(os.path.join(SAVE_PATH, 'images'))

In [24]:
pixel_size = 1.4e-3 # pixel size in mm

transforms = {}
transforms["focal_length"] = 4.74 #Focal Length in mm
transforms["pixel_width"] = 3*pixel_size
transforms["pixel_height"] = transforms["pixel_width"]
transforms["w"] = 1280  # image width
transforms["h"] = 720 # image height

transforms["fov_x"] = 2*math.atan((transforms["w"]*transforms["pixel_width"])/(2*transforms["focal_length"])) # field of view in x
transforms["fl_x"] = transforms["focal_length"]/transforms["pixel_width"] # focal length/ pixel_width
transforms["fl_y"] = transforms["focal_length"]/transforms["pixel_height"]# focal length/ pixel_height
transforms["cx"] = transforms["w"]//2 # optical center in x
transforms["cy"] = transforms["h"]//2 # optical center in y
transforms["camera_model"] = "OPENCV" # camera model

ground_truth = {}
ground_truth = transforms.copy()

In [25]:
print(transforms["pixel_width"])
print(transforms["fov_x"])

0.0042
1.031736666064982


In [26]:
frames = []
ground_truth_frames = []

z = 2.5
pitch = 0.785
total_frames = 300

fov = 2

for cnt in range(total_frames):
    
    r = np.random.uniform(1.5, 4)
    angle = np.random.uniform(0, 2*np.pi)
    
    start_transform_matrix = np.eye(4)
    start_transform_matrix[:3, :3] = Rotation.from_rotvec(np.array([pitch, 0, np.random.uniform(-fov//2,fov//2)])).as_matrix()
    start_transform_matrix[0, 3] = 0
    start_transform_matrix[1, 3] = -r
    start_transform_matrix[2, 3] = z

      
    rot = Rotation.from_rotvec(np.array([0, 0, 1]) * angle).as_matrix()
    rot_matrix = np.eye(4)
    rot_matrix[:3, :3] = rot
    transform_matrix = np.matmul(rot_matrix, start_transform_matrix)

    frame = {}
    file_path = "images/{}.png".format(cnt)

    frame["file_path"] = file_path
    frame["transform_matrix"] = transform_matrix.tolist().copy()
    frames.append(frame.copy())

    frame["transform_matrix"] = transform_matrix.tolist().copy()
    ground_truth_frames.append(frame.copy())

transforms["frames"] = frames
ground_truth["frames"] = ground_truth_frames

In [27]:
print(transforms["frames"])
print(ground_truth["frames"])

[{'file_path': 'images/0.png', 'transform_matrix': [[0.7133381700457433, 0.3859745570796473, -0.584954952490327, -1.937278729071614], [-0.6608007511585782, 0.6484521175706256, -0.3779579586229541, -0.45738860407668364], [0.23343312195155486, 0.6561505105578612, 0.7176179241564475, 2.5000000000000004], [0.0, 0.0, 0.0, 1.0]]}, {'file_path': 'images/1.png', 'transform_matrix': [[0.7051123782624857, 0.4962894859994507, -0.5064714010755419, -2.7247148114176527], [-0.7090148864685292, 0.5042202337877035, -0.49301100049095503, -2.573147354950965], [0.010696952231364493, 0.7067239219988769, 0.7074085617855392, 2.4999999999999996], [0.0, 0.0, 0.0, 1.0]]}, {'file_path': 'images/2.png', 'transform_matrix': [[-0.44351167458382335, 0.5821934388925838, -0.6814309900630289, -3.640908259470728], [-0.8470624818876852, -0.5207307353543468, 0.10641735307550121, -1.0614585720127754], [-0.29288657580388927, 0.6244119641452653, 0.7241043797313506, 2.4999999999999996], [0.0, 0.0, 0.0, 1.0]]}, {'file_path': '

In [28]:
#Save transforms
with open(os.path.join(SAVE_PATH,'transforms.json'), 'w') as outfile:
    json.dump(transforms, outfile, indent=4)
with open(os.path.join(SAVE_PATH,'ground_truth.json'), 'w') as outfile:
    json.dump(ground_truth, outfile, indent=4)